In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
f = open("/kaggle/input/expedia-hotel-recommendations/train.csv", "r")
a = f.readline().strip().split(',')
print(a[5])
print(a[6])
print(a[16])
print(a[21])
print(a[-1])
print(a[-6])

In [ ]:
from collections import defaultdict

d_city_distance = defaultdict(lambda: defaultdict(int))
d_destination = defaultdict(lambda: defaultdict(int))
d_country = defaultdict(lambda: defaultdict(int))
d_overall = defaultdict(int)

n = 0

while(1):
    line = f.readline().strip()
    n += 1
    
    if n % 1000000 == 0:
        print('Read {} lines...'.format(n))
    
    if line == '':
        break
        
    arr = line.split(',')
    
    city = arr[5]
    distance = arr[6]
    destination = arr[16]
    country = arr[21]
    score = 20 if arr[-6] == 1 else 1
    hotel = arr[-1]
    
    if city != '' and distance != '':
        d_city_distance[(city, distance)][hotel] += score
    
    if destination != '':
        d_destination[destination][hotel] += score
    
    if country != '':
        d_country[country][hotel] += score
    
    d_overall[hotel] += score

In [ ]:
f.close()
f2 = open("/kaggle/input/expedia-hotel-recommendations/test.csv","r")
a = f2.readline().split(',')
print(a[0])
print(a[6])
print(a[7])
print(a[17])
print(a[20])

In [ ]:
out = open("submission.csv","w")
out.write("id,hotel_cluster\n")

from heapq import nlargest
from operator import itemgetter

top_overall = nlargest(5, sorted(d_overall.items()), key=itemgetter(1))

n = 0
while(1):
    line = f2.readline().strip()
    n += 1
    
    if n % 100000 == 0:
        print('Read {} lines...'.format(n))
    
    if line == '':
        break
    
    arr = line.split(',')
    
    idx = arr[0]
    city = arr[6]
    distance = arr[7]
    destination = arr[17]
    country = arr[20]
    
    out.write(str(idx) + ',')
    recommend = []
    
    if (city, distance) in d_city_distance:
        hotels_1 = d_city_distance[(city, distance)]
        topitems = nlargest(5, hotels_1.items(), key=itemgetter(1))
        for i in range(len(topitems)):
            if len(recommend) == 5:
                break
            if topitems[i][0] in recommend:
                continue
            out.write(' ' + topitems[i][0])
            recommend.append(topitems[i][0])
            
    if destination in d_destination:
        hotels_2 = d_destination[destination]
        topitems = nlargest(5, hotels_2.items(), key=itemgetter(1))
        for i in range(len(topitems)):
            if len(recommend) == 5:
                break
            if topitems[i][0] in recommend:
                continue
            out.write(' ' + topitems[i][0])
            recommend.append(topitems[i][0])
            
    if country in d_country:
        hotels_3 = d_country[country]
        topitems = nlargest(5, hotels_3.items(), key=itemgetter(1))
        for i in range(len(topitems)):
            if len(recommend) == 5:
                break
            if topitems[i][0] in recommend:
                continue
            out.write(' ' + topitems[i][0])
            recommend.append(topitems[i][0])
            
    for i in range(len(top_overall)):
        if len(recommend) == 5:
            break
        if top_overall[i][0] in recommend:
            continue
        out.write(' ' + top_overall[i][0])
        recommend.append(top_overall[i][0])
    
    out.write('\n')
out.close()